# Data Fetching and Processing Notebook

In [5]:
import pandas as pd
import numpy as np

## Process BERT vec with introduction

Elim the characters in sentences with the Baidu API

In [8]:
raw_dat = pd.read_csv("data/desc.csv", index_col=0)
intro_dat = raw_dat[["ts_code", "introduction", "business_scope", "main_business"]]
intro_dat = intro_dat.fillna("")
intro_dat.loc[:, "intro_all"] = (
    intro_dat.loc[:, "introduction"]
    + intro_dat.loc[:, "business_scope"]
    + intro_dat.loc[:, "main_business"]
)

In [9]:
intro_dat.to_csv('raw_intro.csv')

In [3]:
from aip import AipNlp

APP_ID = "18643028"
API_KEY = "DZ9xbDqRVDhTA5OKO56zwv1k"
SECRET_KEY = "KG59O3eaWioaQdp7fChEOpZAvKF3i14n"

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [4]:
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

SEC_CALL_LIMIT = {"calls": 1, "seconds": 1}


@sleep_and_retry
@limits(calls=SEC_CALL_LIMIT["calls"], period=SEC_CALL_LIMIT["seconds"])
def get_lexer(intro):
    get_code = client.lexer(intro)
    if "error_code" not in get_code:
        return get_code
    else:
        raise Exception("API response: {}".format(get_code))


seperate_dict = {}
for i in tqdm(range(intro_dat.shape[0]), unit="companies"):
    seperate_dict[intro_dat.iloc[i, 0]] = get_lexer(intro_dat.iloc[i, 4])

100%|██████████| 3924/3924 [1:05:29<00:00,  1.00s/companies]


In [5]:
import pickle

f = open("raw_seperate", "wb")
pickle.dump(seperate_dict, f)
f.close()

preprocess word property data queried from Baidu

In [6]:
from tqdm import tqdm


def delete_char(single_dict):
    elim_ticker = ["c", "u", "xc", "w"]
    return True if single_dict["pos"] in elim_ticker else False


def delete_join_char(stock_result_list):
    processed_sentence = ""
    for single_word in stock_result_list:
        if not delete_char(single_word):
            processed_sentence += single_word["item"]
    return processed_sentence

In [11]:
cleaned_dat = pd.DataFrame(columns=["ticker", "intro"])
for comp in tqdm(seperate_dict, desc="Processing intro words", unit="companies"):
    new_sentence = delete_join_char(seperate_dict[comp]["items"])
    new_df = pd.DataFrame([[comp, new_sentence]], columns=["ticker", "intro"])
    cleaned_dat = cleaned_dat.append(new_df, ignore_index=True)

Processing intro words: 100%|██████████| 3924/3924 [00:03<00:00, 1097.63companies/s]


In [12]:
cleaned_dat.head()

,ticker,intro
0,603363.SH,公司是一家以标准化规范化集约化产业化为导向高科技农牧企业公司主营业务包括饲料动保养猪原料贸易...
1,603128.SH,公司是中国本土最具规模提供跨境一站式综合物流服务企业一公司主营跨境现代综合第三方物流具体提供...
2,688039.SH,公司是一家专注于智能视频技术算法研究面向传媒文化公共安全行业国家高新技术企业公司经营模式主要...
3,603127.SH,公司主要从事以药物非临床安全性评价服务为主药物临床前研究服务实验动物附属产品销售业务其中药物...
4,601068.SH,公司是中国铝业集团有限公司工程技术板块是集测绘勘察工程设计工程施工工程总承包科技研发装备制造...


In [20]:
cleaned_dat.to_csv("clean_desc_data.csv")

## Download Price data from Tushare

better to quere with R tidyquant library

In [4]:
import tushare as ts
ts.set_token('1dad4f86f90aa240de1a30832a5e1ff4e13c646685852f4f7ac30832')
pro=ts.pro_api()

In [4]:
cleaned_dat=pd.read_csv('data/clean_desc_data.csv')
short = cleaned_dat[cleaned_dat["intro"].str.len() <= 512]
short['ticker'].to_csv('queuename.csv')

f:\programs\Miniconda3\envs\hejj3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
from ratelimit import limits, sleep_and_retry

SEC_CALL_LIMIT = {"calls": 500, "seconds": 60}

@sleep_and_retry
@limits(calls=SEC_CALL_LIMIT["calls"], period=SEC_CALL_LIMIT["seconds"])
def get_price_data(ticker,start_date,end_date):
    return pro.daily_basic(ts_code=ticker, start_date=start_date, end_date=end_date)

In [ ]:
from tqdm import tqdm
prices_df=[]
start_date='20190630'
end_date='20191231'
for ticker in tqdm(namelist,desc='Downloading prices',unit='companies'):
    prices_df.append(get_price_data(ticker,start_date,end_date))